## Getting a bit more deeper into Agent

### Conversational ReAct Agent

This agent is designed for use in conversational settings. It incorporates the ReAct framework to determine which tool to use and utilizes memory to member previous conversation interactions.

→ <i>Agent này được thiết kế để sử dụng trong đàm thoại. Nó kết ReAct framework để xác định công cụ nào sẽ sử dụng và tận dụng bộ nhớ để ghi lại các tương tác hội thoại trước đó.</i>

In [74]:
import os
from dotenv import load_dotenv
import openai
from langchain_openai import OpenAI, ChatOpenAI
from langchain.agents import Tool, AgentType, initialize_agent, load_tools, AgentExecutor
from langchain.memory import ConversationBufferMemory

In [40]:
# Setup model
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
openai.api_key = api_key

In [41]:
llm = ChatOpenAI(openai_api_key=api_key,model="gpt-3.5-turbo-0125",temperature=0.5)

In [42]:
tools = load_tools(["llm-math"],llm=llm)

In [43]:
memory = ConversationBufferMemory(memory_key="chat_history")
conversational_agent = initialize_agent(
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    tools = tools,
    llm = llm,
    verbose = True,
    memory = memory
)

In [44]:
output_1 = conversational_agent.invoke("Add 7 to 9 and tell me the result")
output_2 = conversational_agent.run("Add 5 to previous result")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Calculator
Action Input: Add 7 to 9
Observation: Answer: 16
Thought:Do I need to use a tool? No
AI: The result of adding 7 to 9 is 16.

> Finished chain.


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Calculator
Action Input: Add 16 to 5
Observation: Answer: 21
Thought:Do I need to use a tool? No
AI: The result of adding 16 to 5 is 21.

> Finished chain.


### DocStore ReAct Agent

This <b>Agent</b> utilizes the <b>ReAct framework</b> to communicate with the docstore. It requires the availability of a <b>Search tool</b> and a <b>Lookup tool</b> with identical names. The Search tool is used to search for a document, while the Lookup tool looks up a term within the most recently found document.

→ <i> Agent này sử dụng ReAct framework để liên lạc với kho tài liệu. Nó yêu cầu có sẵn công cụ Tìm kiếm và công cụ Tra cứu có tên giống hệt nhau. Công cụ Tìm kiếm được sử dụng để tìm kiếm tài liệu, trong khi công cụ Tra cứu tra cứu một thuật ngữ trong tài liệu được tìm thấy gần đây nhất.</i>

In [56]:
from langchain import Wikipedia
from langchain.agents.react.base import DocstoreExplorer

In [84]:
# docstore = DocstoreExplorer(Wikipedia())
tools = [
    Tool(name="Search",func=docstore.search,description="Useful for when you need to search Wikipedia"),
    Tool(name="Lookup",func=docstore.lookup,description="Useful for when you need to lookup a term in Wikipedia"),
]
llm = ChatOpenAI(openai_api_key=api_key,temperature=0)
docstore_agent = initialize_agent(
    agent=AgentType.REACT_DOCSTORE,
    tools = tools,
    llm = llm,
    verbose = True,
    handle_parsing_errors=True,
    return_intermediate_steps=True,
    max_iterations=2
)
docstore_agent.invoke("Tell me few key things about Napoleon")



> Entering new AgentExecutor chain...
Thought: I need to search for information about Napoleon to provide a few key points about him.
Action: Search[Napoleon]
Observation: Napoleon Bonaparte (born Napoleone di Buonaparte; 15 August 1769 – 5 May 1821), later known by his regnal name Napoleon I, was a French emperor and military commander who rose to prominence during the French Revolution and led successful campaigns during the Revolutionary Wars. He was the leader of the French Republic as First Consul from 1799 to 1804, then of the French Empire as Emperor of the French from 1804 until 1814, and briefly again in 1815. His political and cultural legacy endures as a celebrated and controversial leader. He initiated many enduring reforms, but has been criticized for his authoritarian rule. He is considered one of the greatest military commanders in history and his wars and campaigns are still studied at military schools worldwide. However, historians still debate the degree to which he

{'input': 'Tell me few key things about Napoleon',
 'output': 'Agent stopped due to iteration limit or time limit.',
 'intermediate_steps': [(AgentAction(tool='Search', tool_input='Napoleon', log='Thought: I need to search for information about Napoleon to provide a few key points about him.\nAction: Search[Napoleon]'),
   "Napoleon Bonaparte (born Napoleone di Buonaparte; 15 August 1769 – 5 May 1821), later known by his regnal name Napoleon I, was a French emperor and military commander who rose to prominence during the French Revolution and led successful campaigns during the Revolutionary Wars. He was the leader of the French Republic as First Consul from 1799 to 1804, then of the French Empire as Emperor of the French from 1804 until 1814, and briefly again in 1815. His political and cultural legacy endures as a celebrated and controversial leader. He initiated many enduring reforms, but has been criticized for his authoritarian rule. He is considered one of the greatest military c

### Self-Ask Agents

The Self-Ask approach allows an LLM to give answers to a question on which it was not explicitly trained. The model might not have a direct answer to a question, but answers to sub-questions will exist in the LLM dataset. It achieves this by obtaining an Intermediate Answer.

→ <i>Cách tiếp cận Tự hỏi cho phép LLM đưa ra câu trả lời cho một câu hỏi mà nó chưa được đào tạo rõ ràng. Mô hình có thể không có câu trả lời trực tiếp cho một câu hỏi nhưng câu trả lời cho các câu hỏi phụ sẽ tồn tại trong bộ dữ liệu LLM. Nó đạt được điều này bằng cách nhận được Câu trả lời trung gian.</i>

![Self-Ask Agent](self-ask.png)

In [72]:
from langchain.tools import DuckDuckGoSearchRun
search = DuckDuckGoSearchRun()
tools = [
    Tool(
        func=search.run, 
        name="Intermediate Answer",
        description="Useful for when you need to search the Internet for information"
    )
]
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.SELF_ASK_WITH_SEARCH,
    verbose = True,
    handle_parsing_errors=True,
    return_intermediate_steps=True
)
agent.invoke("Question: Who was the US president when the nuclear bomb was first invented? ")



> Entering new AgentExecutor chain...
Yes.
Follow up: When was the nuclear bomb first invented?
Intermediate answer: Manhattan Project, U.S. government research project (1942-45) that produced the first atomic bombs. The project's name was derived from its initial location at Columbia University, where much of the early research was done. The first bomb was exploded in a test at Alamogordo air base in southern New Mexico on July 16, 1945. Manhattan District The Trinity test of the Manhattan Project on 16 July 1945 was the first detonation of a nuclear weapon. Active 1942-1946 Disbanded 15 August 1947 Country United States United Kingdom Canada Branch U.S. Army Corps of Engineers Garrison/HQ Oak Ridge, Tennessee, U.S. Anniversaries 13 August 1942 Engagements Allied invasion of Italy Allied invasion of France Allied invasion of ... J. Robert Oppenheimer is known as the "father of the atomic bomb" for his role in creating the first nuclear weapon during World War II. The theoretical phy

{'input': 'Question: Who was the US president when the nuclear bomb was first invented? ',
 'output': 'Harry S. Truman',
 'intermediate_steps': [(AgentAction(tool='Intermediate Answer', tool_input='When was the nuclear bomb first invented?', log='Yes.\nFollow up: When was the nuclear bomb first invented?'),
   'Manhattan Project, U.S. government research project (1942-45) that produced the first atomic bombs. The project\'s name was derived from its initial location at Columbia University, where much of the early research was done. The first bomb was exploded in a test at Alamogordo air base in southern New Mexico on July 16, 1945. Manhattan District The Trinity test of the Manhattan Project on 16 July 1945 was the first detonation of a nuclear weapon. Active 1942-1946 Disbanded 15 August 1947 Country United States United Kingdom Canada Branch U.S. Army Corps of Engineers Garrison/HQ Oak Ridge, Tennessee, U.S. Anniversaries 13 August 1942 Engagements Allied invasion of Italy Allied inv

C:\Users\ASUS\anaconda3\lib\site-packages\curl_cffi\aio.py:205: UserWarning: Curlm alread closed! quitting from process_data
  warnings.warn("Curlm alread closed! quitting from process_data")
